# Laboratorio 5 Redes Neuronales con Pythorch (Grupo 1)
 <h3>Implementacion del codigo del redes neuronales con pythorch, este modelo busca realizar la prediccion de imagenes de Lenguaje de Señas<h3>
 <HR>
 <h3>
  NOMBRE: Delgadillo LLanos Juan Sebastian <br>
  CARRERA: Ingenieria de Sistemas <BR>
  
  * [Enlace al git hub](https://github.com/sebastianDLL/SIS420_IA/tree/main)
  
  * [Enlace al Colab](https://colab.research.google.com/drive/1FCdaWCkDDFZZzGjhiWusWytvKDVjWw1U?usp=sharing)
 <h3>

# IMPORTAMOS LIBRERIAS PARA EL EJERCICIO

In [ ]:
# utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np
# Libreria para graficos
from matplotlib import pyplot as plt


import torch
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.

import torchvision # torch package for vision related things
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
from tqdm import tqdm  # For nice progress bar!

from torchvision.datasets import ImageFolder

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

# conectamos al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Iniciamos la clase RedNeuronalMLS, definiendo las capas correspondientes



In [ ]:
# Here we create our simple neural network. For more details here we are subclassing and
# inheriting from nn.Module, this is the most general way to create your networks and
# allows for more flexibility. I encourage you to also check out nn.Sequential which
# would be easier to use in this scenario but I wanted to show you something that
# "always" works.
class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__()
        # Our first linear layer take input_size, in this case 160.000 x 3 --> 480000 nodes to 30
        # and our second linear layer takes 30 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)
        """
        x = self.fc1(x)
        x = F.sigmoid(x)
        # x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

Asignamos el dispositivo (GPU o CPU)
Tambien definimos la capa de entrada que seria 480000 porque nuestras imagenes son de 400x400 y son a colores lo que debemos multiplicar por 3

In [ ]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 480000  #400x400x3 --> rgb   400x400 --> grises
num_classes = 10
learning_rate = 0.001
batch_size = 50000
num_epochs = 15

cpu


LEEMOS EL DATASET DE IMAGENES SEPARANDO PARA ENTRENAMIENTO Y PARA PROBAR

(80% - 20%)

In [ ]:
# Definir la transformación para convertir las imágenes a tensores
transformacion = transforms.Compose([
    #transforms.Resize((400, 400)),  # Redimensionar las imágenes a un tamaño específico
    transforms.ToTensor()            # Convertir a tensor
])

# Rutas a las carpetas de entrenamiento y prueba
ruta_train = "/content/drive/MyDrive/SIS420/LABORATORIOS/Lab5/asl_dataset/train"
ruta_test = "/content/drive/MyDrive/SIS420/LABORATORIOS/Lab5/asl_dataset/test"

# Crear conjuntos de datos de imagen para entrenamiento y prueba
train_dataset = ImageFolder(root=ruta_train, transform=transformacion)
test_dataset = ImageFolder(root=ruta_test, transform=transformacion)

# Crear DataLoader para entrenamiento y prueba
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

print("Número de imágenes de entrenamiento:", len(train_dataset))
print("Número de imágenes de prueba:", len(test_dataset))

Número de imágenes de entrenamiento: 560
Número de imágenes de prueba: 140


# **INICIAMOS LA RED NEURONAL**

In [ ]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)
model

RedNeuronalMLS(
  (fc1): Linear(in_features=480000, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([560, 480000])


  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([560, 480000])


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([560, 480000])


  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([560, 480000])


100%|██████████| 1/1 [00:07<00:00,  7.52s/it]


## **VERIFICAMOS EL PORCENTAJE DE EFECTIVIDAD DEL MODELO CON AMBOS GRUPOS PRUEBA Y ENTRENAMIENTO**

In [ ]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Accuracy on training set: {p_train*100:.2f}")
print(f"Accuracy on test set: {p_test*100:.2f}")

Accuracy on training set: 83.04
Accuracy on test set: 53.57
